<a href="https://colab.research.google.com/github/khushbooekhande/DrugRecommendationSystem/blob/master/Implementation/DrugRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd 
import numpy as np

In [0]:
pd.set_option('display.max_columns', None)
df = pd.read_csv(f'/content/drive/My Drive/sentiment_review.csv')
df=df.drop(['Symptoms', 'Rating', 'Unnamed: 0'], axis=1)
df.Drug = df.Drug.str.upper()

In [0]:
predicted_disease = 'osteoporosis'

In [0]:
# function to get drugs based on sentiment review
def _DrugRecommender(df=df, predicted_disease=predicted_disease):
    #df['Rating'] = np.where(df['Rating']>7, 1, 0)
    df = df.loc[df['Disease'] == predicted_disease]
    df = df.loc[df['sentiment_review'] == 'positive']
    df = df.sort_values(by = ['UsefulCount'], ascending=False)
    df = df[['Disease', 'Drug']]
    df = df.drop_duplicates()
    df = df.reset_index()
    df = df.drop(['index'], axis=1)
    df = df.head()

    return df

In [0]:
drug = _DrugRecommender()

In [7]:
print('List of recommened drugs based on useful count and sentiment analysis')
drug

List of recommened drugs based on useful count and sentiment analysis


Disease             Drug
0  osteoporosis        DENOSUMAB
1  osteoporosis           PROLIA
2  osteoporosis     TERIPARATIDE
3  osteoporosis           FORTEO
4  osteoporosis  ZOLEDRONIC ACID

In [0]:
sd = pd.read_csv(f'/content/drive/My Drive/SideEffects_processed_new.csv')
sd = sd.drop(['condition', 'commentsReview'], axis=1)
sd = sd.dropna()

In [0]:
sd.sideEffects = sd.sideEffects.replace('Moderate Side Effects', 2)
sd.sideEffects = sd.sideEffects.replace('Mild Side Effects', 2)
sd.sideEffects = sd.sideEffects.replace('Severe Side Effects', 1)
sd.sideEffects = sd.sideEffects.replace('No Side Effects', 3)
sd.sideEffects = sd.sideEffects.replace('Extremely Severe Side Effects', 0)

In [0]:
sd.effectiveness = sd.effectiveness.replace('Considerably Effective', 2)
sd.effectiveness = sd.effectiveness.replace('Highly Effective', 3)
sd.effectiveness = sd.effectiveness.replace('Marginally Effective', 1)
sd.effectiveness = sd.effectiveness.replace('Moderately Effective', 1)
sd.effectiveness = sd.effectiveness.replace('Ineffective', 0)

In [0]:
sd = sd[['urlDrugName', 'rating', 'effectiveness', 'sideEffects', 'sideEffectsReview']]

In [0]:
sd = sd.rename(columns={"urlDrugName": "Drug", "rating": "Rating", "effectiveness": "Effectivness Rating",
                       "sideEffects": "Side Effect Rating", "sideEffectsReview": "Side Effects"})
sd.Drug = sd.Drug.str.upper()

In [14]:
print('Drug with possible side effects')
sd

Drug with possible side effects


Drug  Rating  Effectivness Rating  Side Effect Rating  \
0            ENALAPRIL       4                    3                   2   
1     ORTHO-TRI-CYCLEN       1                    3                   1   
2              PONSTEL      10                    3                   3   
3             PRILOSEC       3                    1                   2   
4               LYRICA       2                    1                   1   
...                ...     ...                  ...                 ...   
4144         DENOSUMAB       8                    2                   3   
4145            PROLIA      10                    3                   3   
4146      TERIPARATIDE       7                    1                   2   
4147            FORTEO       7                    1                   2   
4148   ZOLEDRONIC ACID       6                    1                   2   

                                           Side Effects  
0     cough, hypotension , proteinuria, impotence , ...  
1     Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...  
2            Heavier bleeding and clotting than normal.  
3     Constipation, dry mouth and some mild dizzines...  
4     I felt extremely drugged and dopey.  Could not...  
...                                                 ...  
4144  Back pain, blistering, crusting, cracked, dry ...  
4145  Back pain, blistering, crusting, cracked, dry ...  
4146  abdominal pain, confusion, constipation, depre...  
4147  abdominal pain, confusion, constipation, depre...  
4148  Agitation, blurred vision, cough, depression, ...  

[4147 rows x 5 columns]

In [0]:
def _probScore(df, w1, w2, w3):
    return(1-(df[w1] * df[w2] * df[w3]).sum()/df[w1].sum()/10)

In [0]:
def _getSE(df, sedf, l):
    sdf = sedf
    sdf = sdf.loc[sdf['Drug'] == l]
    #print(sdf)
    w = sdf.groupby(["Drug"]).apply(_probScore, "Rating", "Effectivness Rating", "Side Effect Rating")
    sdf1 = sdf.loc[sdf['Effectivness Rating'] == 0]
    sdf1 = sdf1.loc[sdf['Side Effect Rating'] == 0]
    sdf1 = sdf1.loc[sdf['Rating'] <= 1 ]
    sdf1 = sdf1.reset_index()
    sdf1 = pd.DataFrame(sdf['Side Effects'])
    df1 = pd.DataFrame(w, columns=['Prob. of Side Effect'])
    df1 = df1.reset_index()
    #print(sdf[:1]['Side Effects'])
    df2 = pd.DataFrame(sdf['Side Effects'])
    df2 = df2.reset_index()
    df2 = df2.drop(['index'], axis=1)
    dd = pd.concat([df1, df2], axis=1)
    
    return dd

In [0]:
def _getSideEffects(df, sedf):
    l = list(df.Drug)
    #print(l)
    for i in range(0, 1):
        df1 = _getSE(df, sedf, l[i])
    dd = df1
    for i in range(1, len(l)):
        df2 = _getSE(df, sedf, l[i])
        dd = pd.concat([dd, df2])
    return dd

In [0]:
se = _getSideEffects(drug, sd)

In [19]:
recommender = drug.set_index('Drug').join(se.set_index('Drug'))
recommender = recommender.reset_index()
recommender['Prob. of Side Effect'] = recommender['Prob. of Side Effect'].fillna(0)
recommender['Prob. of Side Effect'] = recommender['Prob. of Side Effect'].fillna('Not Available')
print('Mapped recommended drugs with possible side effects and probabilistic score')
recommender

Mapped recommended drugs with possible side effects and probabilistic score


Drug       Disease  Prob. of Side Effect  \
0        DENOSUMAB  osteoporosis                   0.4   
1           PROLIA  osteoporosis                   0.1   
2     TERIPARATIDE  osteoporosis                   0.8   
3           FORTEO  osteoporosis                   0.8   
4  ZOLEDRONIC ACID  osteoporosis                   0.8   

                                        Side Effects  
0  Back pain, blistering, crusting, cracked, dry ...  
1  Back pain, blistering, crusting, cracked, dry ...  
2  abdominal pain, confusion, constipation, depre...  
3  abdominal pain, confusion, constipation, depre...  
4  Agitation, blurred vision, cough, depression, ...

In [20]:
#sorting based on probabilistic score
recommender = recommender.sort_values('Prob. of Side Effect')
print("The recommended Drugs for the given Disease is:")
recommender

The recommended Drugs for the given Disease is:


Drug       Disease  Prob. of Side Effect  \
1           PROLIA  osteoporosis                   0.1   
0        DENOSUMAB  osteoporosis                   0.4   
2     TERIPARATIDE  osteoporosis                   0.8   
3           FORTEO  osteoporosis                   0.8   
4  ZOLEDRONIC ACID  osteoporosis                   0.8   

                                        Side Effects  
1  Back pain, blistering, crusting, cracked, dry ...  
0  Back pain, blistering, crusting, cracked, dry ...  
2  abdominal pain, confusion, constipation, depre...  
3  abdominal pain, confusion, constipation, depre...  
4  Agitation, blurred vision, cough, depression, ...